<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Проект-для-«Викишоп»" data-toc-modified-id="Проект-для-«Викишоп»-1">Проект для «Викишоп»</a></span><ul class="toc-item"><li><span><a href="#1.-Загрузка-данных" data-toc-modified-id="1.-Загрузка-данных-1.1">1. Загрузка данных</a></span></li><li><span><a href="#2.-Подготовка-данных" data-toc-modified-id="2.-Подготовка-данных-1.2">2. Подготовка данных</a></span></li><li><span><a href="#3.-Обучение" data-toc-modified-id="3.-Обучение-1.3">3. Обучение</a></span><ul class="toc-item"><li><span><a href="#3.1.-Обучение-моделей." data-toc-modified-id="3.1.-Обучение-моделей.-1.3.1">3.1. Обучение моделей.</a></span></li><li><span><a href="#3.2.-Анализ-моделей." data-toc-modified-id="3.2.-Анализ-моделей.-1.3.2">3.2. Анализ моделей.</a></span></li></ul></li><li><span><a href="#4.-Вывод" data-toc-modified-id="4.-Вывод-1.4">4. Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-1.5">Чек-лист проверки</a></span></li></ul></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель**

Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо постройть модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.


 ## 1. Загрузка данных

In [ ]:
import re
import nltk
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV




warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') 
stopwords = set(nltk_stopwords.words('english'))

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
def check_df(df):    
    display(df.info())   
    display(df.head())
    display(df.describe())
    display(df.isna().sum())   
    display(df.duplicated().sum())

In [ ]:
check_df(toxic_comments)

**Вывод**
1. Всего имеет 159 292 строк. 
2. Названия колонок в нужном регистре. При этом текст в строках в разных. Нужно будет привести к единому регистру.
3. Тип данных соответсвует данным. 
4. Пропусков и явных дубликатов нет.
5. В тексте есть лишние символы типа "\n и т.д, от которых необходимо избавиться.
6. Колонку Unnamed, повторяющая индексы, необходимо будет удалить.

## 2. Подготовка данных

Удалим столбец Unnamed.

In [ ]:
toxic_comments = toxic_comments.drop(['Unnamed: 0'], axis=1)
toxic_comments.info()

Приведем текст к нижнему регистру.

In [ ]:
toxic_comments['text'] = toxic_comments['text'].str.lower()
toxic_comments.head()

Создадим функцию для очистки текста.

In [ ]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [ ]:
toxic_comments['text'] = toxic_comments['text'].apply(clear_text)

In [ ]:
toxic_comments.head()

Текст очистили от лишних символов. Теперь необходимо лемматизировать текст. Воспользуемся функцией для POS-тегирования слов, которая возвращает части речи, а затем лемматизируем при помощи WordNetLemmatizer().

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return " ".join(text)

In [ ]:
toxic_comments['lemm_text'] = toxic_comments['text'].apply(lemmatize)

In [ ]:
toxic_comments .head()

Удалим колонку text.

In [ ]:
toxic_comments = toxic_comments.drop(['text'], axis=1)
toxic_comments.head()

Проверим соотношение отзывов.

In [ ]:
display(toxic_comments['toxic'].value_counts())

Незативных отзывов гораздо больше. Разобьем на тренировочную и тестовую выборки.

In [ ]:
features = toxic_comments.drop(['toxic'], axis=1) 
target = toxic_comments['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, 
    target,
    stratify=target,
    test_size=.25, 
    random_state=42
) 

print('features_train:', features_train.shape)
print('target_train:', features_test.shape)
print('features_test:',target_train.shape)
print('target_test:', target_test.shape)

**Вывод:** 
на данном этапе
1. Очистили текст от лишних символов
2. Лемматизировали текст
3. Разбили на выборки


## 3. Обучение

Обучим несколько моделей, с использованием кросс-валидации. Для обучения были выбраны модели LogisticRegression, DecisionTreeClassifier, SGDClassifier.

### 3.1. Обучение моделей.

Создадим функцию для обучения модели, используя паплайн и кросс-валидацию. В паплайн добавим TfidfVectorizer.

In [ ]:
data_cv = []

def model_search(
    features_train, 
    target_train, 
    model, 
    parameters, 
    data_cv
):
    

    pipeline = Pipeline(
        [
            ('vect', ColumnTransformer([
                ('tfidf', TfidfVectorizer(), 'lemm_text')
        ], remainder='drop')
            ),
            ('model', model) 
    ])
    
    model_cv = HalvingGridSearchCV(
        pipeline, 
        parameters, 
        cv=4,
        n_jobs=-1, 
        scoring='f1', 
        error_score='raise',
        random_state=42
    ) 
    
    model_cv.fit(features_train, target_train)
    
    data_cv.append(model_cv)
 
    
    return data_cv

In [ ]:
def result(data, model_name):
    print('Модель:', model_name)
    print('f1 для модели:', round(data[-1].best_score_, 3))
    print('Гиперпараметры', data[-1].best_params_)

In [ ]:
parameters = [{
    'model__penalty': ['none', 'l2']
}]

data_cv = model_search(
    features_train, 
    target_train,  
    LogisticRegression(random_state=42), 
    parameters, 
    data_cv)

result(data_cv, 'LogisticRegression')

In [ ]:
parameters = [{
    'model__max_depth' : range(3, 16, 3),
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}]

data_cv = model_search(
    features_train, 
    target_train, 
    DecisionTreeClassifier(random_state=42),
    parameters, 
    data_cv)

result(data_cv, 'DecisionTreeClassifier')

In [ ]:
parameters = [{
    'model__loss': ['hinge', 'log', 'modified_huber'],
    'model__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'model__eta0': [.01, .05, .1]
}]

data_cv = model_search(
    features_train, 
    target_train, 
    SGDClassifier(random_state=42),
    parameters, 
    data_cv)

result(data_cv, 'SGDClassifier')

**Вывод:** на данном этапе обучили три модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier при помощи паплайна. Перейдем к анализу моделей.

### 3.2. Анализ моделей.

Выберем лучшую модель.

In [ ]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'SGDClassifier']

data = {'f1_cv':['0.762',
                '0.638',
                '0.748']
       }

df = pd.DataFrame(data=data, index=index)
df

In [ ]:
model_best = data_cv[0]

n = 0
for i in range(0, len(data_cv)):
    if data_cv[i].best_score_ > model_best.best_score_:  
        data_grids_best = data_grids[i]

print('Лучшая метрика f1:', model_best.best_score_)
print('Лучшая модель: ')
model_best

Модель LogisticRegression справляется лучше всех. Посчитаем метрику на тестовых данных для лучшей модели.

In [ ]:
model_predict = model_best.predict(features_test)


print(f'f1 на тестовой выборке для модели LogisticRegression :{round(f1_score(target_test, model_predict), 3)}')

**Вывод:** на данном этапе провели анализ трех моделей LogisticRegression, DecisionTreeClassifier, RandomForestClassifier. Лучшая модель LogisticRegression с метрикой f1 на тренировочных данных 0,76, на тестовых 0, 77.

## 4. Вывод

Был получен датасет из 159292 строк. Данные были подготовлены: очистили и лемматизировали текст, далее разбили на тренирвочные и тестовые выборки. Провели обучение трех моделей. Лучшей моделью оказалась LogisticRegression с метрикой f1 на тренировочных данных 0,76, на тестовых 0, 77. после взвешивания классов модели.